
# Workflow Practice

In this notebook, you’ll practice connecting to a SQLite database, creating tables from CSV files using Pandas, and writing SQL queries to explore the data.

The dataset comes from the [Bike Store Sample Database](https://www.kaggle.com/datasets/dillonmyrick/bike-store-sample-database) by Dillon Myrick. It models a fictional bike retailer with multiple stores, products, customers, and staff. Each table connects to others using foreign keys such as `customer_id`, `store_id`, and `product_id`.

You’ll:
- Connect to a local SQLite database
- Create tables using `pandas.to_sql()`
- Write and test SQL queries using `pd.read_sql()`

All of your work will take place directly in this notebook. Each question prompt is written below as a Markdown cell, followed by an empty code cell for you to write your query.



## Step 1: Connect to the Database

Run the following cell to connect to (or create) a SQLite database called `bike_store.db`.  
If the file doesn’t exist yet, SQLite will automatically create it.


In [1]:
import sqlite3
import pandas as pd

In [2]:
connection = sqlite3.connect("bike_store.db")
connection


## Step 2: Create Tables from CSV Files

The `data/` folder contains one CSV file per table.  
Use `pandas.read_csv()` and `DataFrame.to_sql()` to load each file into your database.

You only need to do this once.  
After that, you’ll be able to run queries against your newly created tables.


In [3]:
# Example for one file
customers = pd.read_csv("data/customers.csv")
customers.to_sql("customers", connection, if_exists="replace", index=False)

1445

In [21]:
# Repeat for all other files in the data folder, or use a loop.

import os 
csv_files = []
for file in os.listdir('data'):
    if file.endswith('.csv') and csv_files.count(file) < 1:
        csv_files.append(file)
        
for table_name in csv_files:
    file_path = pd.read_csv(f'data/{table_name}')
    raw_name = table_name.removesuffix('.csv')
    file_path.to_sql(raw_name, connection, if_exists='replace', index=False)


### Verify Your Tables

Run a query to make sure your tables were created successfully.

In [22]:

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)


,name
0,customers
1,categories
2,products
3,orders
4,staffs
5,order_items
6,brands
7,stores
8,stocks


## Step 3: Test a Simple Query

Before starting the exercises, confirm your connection and tables are working by previewing the first few rows of the `customers` table.

In [ ]:

pd.read_sql("SELECT * FROM customers LIMIT 5;", connection)


### Q1. List all customers and their cities.

Return the first name, last name, and city of each customer. Sort alphabetically by last name and then by first name.

In [29]:
# Your query here
pd.read_sql('SELECT first_name, last_name, city FROM customers ORDER BY last_name', connection)

pd.read_sql('SELECT first_name, last_name, city FROM customers ORDER BY first_name', connection)


,first_name,last_name,city
0,Aaron,Knapp,Yonkers
1,Abbey,Pugh,Forest Hills
2,Abby,Gamble,Amityville
3,Abram,Copeland,Harlingen
4,Adam,Henderson,Los Banos
...,...,...,...
1440,Zona,Cameron,East Northport
1441,Zora,Ford,Sacramento
1442,Zoraida,Patton,Huntington Station
1443,Zulema,Browning,Amsterdam


### Q2. Show all products and their prices.

Display each product name along with its list price. Sort by price in descending order.

In [32]:
# Your query here

pd.read_sql('SELECT product_name, list_price FROM products ORDER BY list_price DESC', connection)

,product_name,list_price
0,Trek Domane SLR 9 Disc - 2018,11999.99
1,Trek Domane SLR 8 Disc - 2018,7499.99
2,Trek Silque SLR 8 Women's - 2017,6499.99
3,Trek Domane SL Frameset - 2018,6499.99
4,Trek Domane SL Frameset Women's - 2018,6499.99
...,...,...
316,Trek Kickster - 2018,159.99
317,Trek Boy's Kickster - 2015/2017,149.99
318,Trek Girl's Kickster - 2017,149.99
319,Sun Bicycles Lil Kitt'n - 2017,109.99


### Q3. Find all customers from California.

Return first name, last name, city, and state for all customers whose state is 'CA'. Sort alphabetically by last name.

In [38]:
# Your query here

pd.read_sql("SELECT first_name, last_name, city, state FROM customers WHERE state='CA' ORDER BY last_name DESC", connection)

,first_name,last_name,city,state
0,Ollie,Zimmerman,Anaheim,CA
1,Yvone,Yates,San Pablo,CA
2,Joel,Wynn,San Diego,CA
3,Lucy,Woods,Palos Verdes Peninsula,CA
4,Darren,Witt,Coachella,CA
...,...,...,...,...
279,Selene,Austin,Duarte,CA
280,Twana,Arnold,Anaheim,CA
281,Sindy,Anderson,Pomona,CA
282,Jamaal,Albert,Torrance,CA


### Q4. Count how many products are in each category.

Return the category name and the number of products in that category. Sort from the highest count to the lowest.

In [43]:
# Your query here

pd.read_sql("""
    SELECT categories.category_name, 
    COUNT(products.category_id) AS product_count
    FROM categories 
    INNER JOIN products 
    ON categories.category_id = products.category_id
    GROUP BY categories.category_name
    ORDER BY COUNT(products.category_id) DESC
""", connection)

,category_name,product_count
0,Cruisers Bicycles,78
1,Road Bikes,60
2,Mountain Bikes,60
3,Children Bicycles,59
4,Comfort Bicycles,30
5,Electric Bikes,24
6,Cyclocross Bicycles,10


### Q5. Find all orders placed in 2018.

List the order ID, order date, and customer ID for orders made during the year 2018. Sort by order date.

In [45]:
# Your query here
pd.read_sql("""
    SELECT
    orders.order_date, 
    orders.order_id,
    orders.customer_id
    FROM orders
    WHERE strftime('%Y', shipped_date)='2018' 
          
""", connection)


,order_date,order_id,customer_id
0,2017-12-31,1322,39
1,2017-12-31,1323,273
2,2018-01-01,1324,862
3,2018-01-01,1325,68
4,2018-01-01,1326,567
...,...,...,...
151,2018-03-30,1474,816
152,2018-03-30,1475,112
153,2018-03-31,1476,35
154,2018-03-31,1477,180


### Q6. Show each order with its total number of items.

Join the `orders` and `order_items` tables. Group by order ID and return the number of items per order.

In [46]:
# Your query here

pd.read_sql("""
    SELECT
    COUNT(order_items.item_id) AS item_count
FROM orders
INNER JOIN order_items
    ON orders.order_id = order_items.order_id
GROUP BY orders.order_id
""", connection)

,item_count
0,5
1,2
2,2
3,1
4,3
...,...
1610,3
1611,5
1612,2
1613,3


### Q7. List total revenue per store.

Revenue = quantity * list_price * (1 - discount). Join `orders`, `order_items`, and `stores`, group by store name, and return total revenue.

In [47]:
# Your query here
pd.read_sql("""
    SELECT
    stores.store_name,
    (order_items.quantity * order_items.list_price) AS total_revenue
    FROM orders
    INNER JOIN order_items
        ON orders.order_id = order_items.order_id
    INNER JOIN stores
        ON orders.store_id = stores.store_id
    GROUP BY stores.store_name
            
""", connection)

,store_name,total_revenue
0,Baldwin Bikes,599.99
1,Rowlett Bikes,3361.98
2,Santa Cruz Bikes,599.99


### Q8. Find the top 5 customers who spent the most overall.

Join `customers`, `orders`, and `order_items`. Sum the total spending per customer and return the top five spenders.

In [ ]:
# Your query here
pd.read_sql("""
    SELECT
    SUM(order_items.list_price)
    FROM customers
    INNER JOIN orders
        ON customers.customer_id = orders.customer_id
    INNER JOIN order_items
        ON orders.order_id = order_items.order_id
    GROUP BY customers.customer_id
    ORDER BY order_items.list_price DESC
    LIMIT 5;
""", connection)


,SUM(order_items.list_price)
0,6499.99
1,5499.99
2,5499.99
3,5499.99
4,11299.98


### Q9. Show the best-selling product in each category.

Join `products`, `order_items`, and `categories`. For each category, identify the product with the highest total quantity sold.

In [53]:
# Your query here
pd.read_sql("""
    SELECT
    categories.category_name,
    SUM(order_items.quantity)
    FROM products
    INNER JOIN order_items
        ON products.product_id = order_items.product_id
    INNER JOIN categories
        ON products.category_id = categories.category_id
    GROUP BY products.category_id
""", connection)

,category_name,SUM(order_items.quantity)
0,Children Bicycles,1179
1,Comfort Bicycles,813
2,Cruisers Bicycles,2063
3,Cyclocross Bicycles,394
4,Electric Bikes,315
5,Mountain Bikes,1755
6,Road Bikes,559


### Q10. Identify the employees (staff) who processed the most orders.

Join `staffs` and `orders`. Count the number of orders handled by each staff member and return the results sorted by highest total.

In [55]:
# Your query here
pd.read_sql("""
    SELECT
    staffs.first_name,
    staffs.last_name,
    COUNT(order_id) AS order_count
    FROM orders
    INNER JOIN staffs
        ON orders.staff_id = staffs.staff_id
    GROUP BY staffs.staff_id
""", connection)

,first_name,last_name,order_count
0,Mireya,Copeland,164
1,Genna,Serrano,184
2,Marcelene,Boyer,553
3,Venita,Daniel,540
4,Kali,Vargas,88
5,Layla,Terrell,86
